In [ ]:
!pip install weaviate-client

In [ ]:
!pip install langchain

ERROR: Operation cancelled by user


In [ ]:
!pip install -U langchain-community

In [ ]:
import weaviate


WEAVIATE_CLUSTER="https://nbwrdgx9taox7s29rhczg.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_API_KEY="89a8hqQ33CPO3MifKQO5UZsnML0qSpDG"


WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY


HF_TOKEN="hf_GcWSgPWmYOvjQpQODvTfLSCZVHpvB"


import os


client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY),
    additional_headers={
         "X-HuggingFace-Api-Key": HF_TOKEN
    },
)

<ipython-input-27-3376394defb9>:18: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


In [ ]:
client.is_ready()

True

In [ ]:
schema = {
    "classes": [
        {
            "class": "RAG",
            "description": "Documents for RAG",
            "vectorizer": "text2vec-huggingface",
            "moduleConfig": {"text2vec-huggingface": {"model": "sentence-transformers/all-MiniLM-L6-v2", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-huggingface": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}


In [ ]:
client.schema.create(schema)

UnexpectedStatusCodeError: Create class! Unexpected status code: 422, with response body: {'error': [{'message': 'class name RAG already exists'}]}.

In [ ]:
client.schema.get()

In [ ]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [ ]:
retriever = WeaviateHybridSearchRetriever(
    alpha = 0.5,               # defaults to 0.5, which is equal weighting between keyword and semantic search
    client = client,           # keyword arguments to pass to the Weaviate client
    index_name = "RAG",  # The name of the index to use
    text_key = "content",         # The name of the text key to use
    attributes = [], # The attributes to return in the results
    create_schema_if_missing=True,
)

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
!pip install bitsandbytes

In [ ]:
!pip install accelerate

In [ ]:
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline

In [ ]:
# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

In [ ]:
# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer


tokenizer = initialize_tokenizer(model_name)


model = load_quantized_model(model_name)

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    #max_length=2048,
    do_sample=True,
    top_k=5,
    max_new_tokens=100,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)


doc_path="/content/NLP for RAG.pdf"


!pip install pypdf

In [ ]:
!pip install langchain_community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader(doc_path)


docs = loader.load()

In [ ]:
retriever.add_documents(docs)

In [ ]:
print(retriever.invoke("what is RAG token?")[0].page_content)

In [ ]:
retriever.invoke(
    "what is RAG token?",
    score=True
)


In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use three sentence maximum and keep the answer concise. "
    "Context: {context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{query}"),
    ]
)


from langchain.prompts import PromptTemplate
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you do not have the relevant information needed to provide a verified answer, don't try to make up an answer.
When providing an answer, aim for clarity and precision. Position yourself as a knowledgeable authority on the topic, but also be mindful to explain the information in a manner that is accessible and comprehensible to those without a technical background.
Always say "Do you have any more questions pertaining to this instrument?" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain


question_answer_chain = create_stuff_documents_chain(llm, prompt)


hybrid_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever,)



result1 = hybrid_chain.invoke("what is natural language processing?")
print(result1)

In [ ]:
query="What is Abstractive Question Answering?"


response = hybrid_chain.invoke({"query":query})

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


# Set up the RAG chain
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} |
    prompt |
    llm
)


query="what is RAG token?"


response=rag_chain.invoke("what is RAG token?")

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank


!pip install cohere

In [ ]:
compressor = CohereRerank(cohere_api_key="MmdZqsmc8Vi4YDbccO1U8NAwdoSnzMomouI")


compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
    )


compressed_docs = compression_retriever.get_relevant_documents(user_query)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)



hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=compression_retriever
)


response = hybrid_chain.invoke("What is Abstractive Question Answering?")

In [ ]:
print(response.get("result"))